<a href="https://colab.research.google.com/github/davidf22/Inventoryanalysis1/blob/main/Redistribute2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df2 = pd.read_csv("products.csv", on_bad_lines='skip')

In [ ]:
df2 = pd.read_csv('products.csv', sep=';', quotechar='"')

In [ ]:
print("\nData types in the DataFrame:")
print(df2.dtypes)


Data types in the DataFrame:
productCode            object
productName            object
productLine            object
productScale           object
productVendor          object
productDescription     object
quantityInStock         int64
warehouseCode          object
buyPrice              float64
MSRP                  float64
dtype: object


In [ ]:
# Total number of all items in inventory (across all warehouses)
total_inventory = df2["quantityInStock"].sum()
print("Total number of stock units (inventory):", total_inventory)


Total number of stock units (inventory): 450086


In [ ]:
df3 = pd.read_csv('orderdetails.csv', sep=';', quotechar='"')

In [ ]:
print("\nData types in the DataFrame:")
print(df3.dtypes)


Data types in the DataFrame:
orderNumber          int64
productCode         object
quantityOrdered      int64
priceEach          float64
orderLineNumber      int64
dtype: object


In [ ]:
df4 = df3[df3["productCode"].isin(df2["productCode"])]


In [ ]:




product_codes_inventory = set(df2["productCode"])
product_codes_orders = set(df4["productCode"])

# Find any codes in orders that are not in inventory
missing_codes = product_codes_orders - product_codes_inventory

if missing_codes:
    print("❌ These product codes in orders do NOT exist in inventory (df2):")
    print(missing_codes)
else:
    print("✅ All product codes in order details exist in inventory.")


✅ All product codes in order details exist in inventory.


In [ ]:
# Step 1: Total quantity ordered (from orderdetails.csv)
total_ordered = df3["quantityOrdered"].sum()
print("🧾 Total quantity ordered:", total_ordered)

🧾 Total quantity ordered: 105516


In [ ]:
df5 = pd.read_csv('orders.csv', sep=';', quotechar='"')

In [ ]:
print("\nData types in the DataFrame:")
print(df5.dtypes)


Data types in the DataFrame:
orderNumber        int64
orderDate         object
requiredDate      object
shippedDate       object
status            object
comments          object
customerNumber     int64
dtype: object


In [ ]:
order_number_inventory = set(df5["orderNumber"])
order_number_details = set(df4["orderNumber"])

# Find any codes in orders that are not in inventory
missing_orders = order_number_details -order_number_inventory

if missing_codes:
    print("❌ These order numbers in orders do NOT exist in inventory (df4):")
    print(missing_codes)
else:
    print("✅ All order numbers in order details exist in inventory.")

✅ All order numbers in order details exist in inventory.


In [ ]:
# Count how many orders fall into each status category
status_counts = df5["status"].value_counts()

print("📦 Order status breakdown:")
print(status_counts)


📦 Order status breakdown:
status
Shipped       303
Cancelled       6
In Process      6
Resolved        4
On Hold         4
Disputed        3
Name: count, dtype: int64


In [ ]:
df6 = pd.read_csv('payments.csv', sep=';', quotechar='"')

In [ ]:
print("First 10 Rows:")
print(df6.head(), "\n")

print(" Data Info:")
print(df6.info(), "\n")

print(" Statistical Summary:")
print(df6.describe(), "\n")


First 10 Rows:
   customerNumber checkNumber paymentDate    amount
0             103    HQ336336  2004-10-19   6066.78
1             103    JM555205  2003-06-05  14571.44
2             103    OM314933  2004-12-18   1676.14
3             112    BO864823  2004-12-17  14191.12
4             112     HQ55022  2003-06-06  32641.98 

 Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 273 entries, 0 to 272
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   customerNumber  273 non-null    int64  
 1   checkNumber     273 non-null    object 
 2   paymentDate     273 non-null    object 
 3   amount          273 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 8.7+ KB
None 

 Statistical Summary:
       customerNumber         amount
count      273.000000     273.000000
mean       271.194139   32431.645531
std        120.069507   20997.116917
min        103.000000     615.450000
25%      

In [ ]:
paid_customers = set(df6["customerNumber"])
shipped_orderss = set(df5["customerNumber"])

# Find any codes in orders that are not in inventory
missing_orders = paid_customers - shipped_orderss

if missing_orders:
    print("❌ These order numbers in orders do NOT exist in inventory (df6):")
    print(missing_orders)
else:
    print("✅ All order numbers in order details exist in inventory.")

✅ All order numbers in order details exist in inventory.


In [ ]:
shipped_orders = df5[df5["status"] == "Shipped"]
print("Total shipped orders:", len(shipped_orders))




Total shipped orders: 303


In [ ]:
shipped_customers = shipped_orders["customerNumber"].unique()

In [ ]:
print(shipped_customers[:5])


['363' '128' '181' '121' '141']


In [ ]:
df5["customerNumber"] = df5["customerNumber"].astype(str).str.strip()
df6["customerNumber"] = df6["customerNumber"].astype(str).str.strip()


In [ ]:
print(df6.columns)


Index(['customerNumber', 'checkNumber', 'paymentDate', 'amount'], dtype='object')


In [ ]:
# Customers who had shipped orders
shipped_customers = df5[df5["status"] == "shipped"]["customerNumber"].unique()

# Check how many of these exist in df6
matching_customers = df6[df6["customerNumber"].isin(shipped_customers)]

print("✅ Customers in df6 with shipped orders:", matching_customers["customerNumber"].nunique())
print(matching_customers.head())


✅ Customers in df6 with shipped orders: 0
Empty DataFrame
Columns: [customerNumber, checkNumber, paymentDate, amount]
Index: []


In [ ]:
# Filter payment records for those customers
shipped_payments = df6[df6["customerNumber"].isin(shipped_customers)]

In [ ]:
print(shipped_payments[:5])

  customerNumber checkNumber paymentDate    amount
0            103    HQ336336  2004-10-19   6066.78
1            103    JM555205  2003-06-05  14571.44
2            103    OM314933  2004-12-18   1676.14
3            112    BO864823  2004-12-17  14191.12
4            112     HQ55022  2003-06-06  32641.98


BREAK

In [ ]:
customer_payments = shipped_payments.groupby("customerNumber")["amount"].sum().reset_index()

In [ ]:
total_paid_shipped = customer_payments["amount"].sum()

In [ ]:
print("💳 Total amount paid by customers with shipped orders:", total_paid_shipped)
print("\n📋 Breakdown per customer:")
print(customer_payments [:5])

💳 Total amount paid by customers with shipped orders: 8853839.23

📋 Breakdown per customer:
  customerNumber     amount
0            103   22314.36
1            112   80180.98
2            114  180585.07
3            119  116949.68
4            121  104224.79


In [ ]:
shipped_orders = df5[df5["status"] == "Shipped"]
shipped_order_numbers = shipped_orders["orderNumber"].unique()


In [ ]:
shipped_quantities = df3[df3["orderNumber"].isin(shipped_order_numbers)]


In [ ]:
print(shipped_quantities[["orderNumber", "quantityOrdered"]].head())


   orderNumber  quantityOrdered
0        10100               30
1        10100               50
2        10100               22
3        10100               49
4        10101               25


In [ ]:
total_shipped_quantity = shipped_quantities["quantityOrdered"].sum()
print("Total quantity of shipped items:", total_shipped_quantity)


Total quantity of shipped items: 97141


In [ ]:
# Total ordered quantity per product
ordered_qty = df4.groupby("productCode")["quantityOrdered"].sum().reset_index()

# Inventory stock per product
stock_qty = df2[["productCode", "quantityInStock"]]

# Merge to align stock and orders
product_demand_supply = pd.merge(ordered_qty, stock_qty, on="productCode", how="left")

# Optional: Calculate surplus or shortage
product_demand_supply["surplus"] = product_demand_supply["quantityInStock"] - product_demand_supply["quantityOrdered"]

print(product_demand_supply.head())


  productCode  quantityOrdered  quantityInStock  surplus
0    S10_1678             1057             7933     6876
1    S10_1949              961             7305     6344
2    S10_2016              999             6625     5626
3    S10_4698              985             5582     4597
4    S10_4757             1030             3252     2222


In [ ]:
overstocked = product_demand_supply[product_demand_supply["surplus"] > 7767]
understocked = product_demand_supply[product_demand_supply["surplus"] < 2824]

print("🔺 Overstocked products:", overstocked.shape[0])
print("🔻 Understocked products:", understocked.shape[0])


🔺 Overstocked products: 10
🔻 Understocked products: 28
